In [111]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import timeit

# Scrape online data

In [3]:
URL = 'https://www.beeradvocate.com/beer/profile/2958/87869/'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')


In [50]:
res = soup.find(id='info_box')

In [51]:
print(res.prettify())

<div id="info_box" style="padding:5px 5px 0px 0px;">
 <div id="main_pic_norm" style="text-align:center; float:right; width:150px; padding:0px; margin:65px 5px 25px 25px;">
  <div style="position:relative;">
   <img alt="XV (âˆš225 Saison)" border="0" src="https://cdn.beeradvocate.com/im/beers/87869.jpg" style="width:100%;"/>
   <span style="position:absolute;left:0;top:0;">
    <img alt="XV (âˆš225 Saison)" border="0" src="https://cdn.beeradvocate.com/im/c_beer_image.gif" style="width:100%;"/>
   </span>
  </div>
 </div>
 <a class="text Tooltip" href="/community/login/?redirect=%2Fbeer%2Fprofile%2F2958%2F87869%2F" style="background:#cc8a00; color:#ffffff; border-radius:5px; padding:5px 8px 5px 5px; text-decoration:none;" title="Submit a new rating or write a review">
  <span>
   <i class="fas fa-check fa-lg">
   </i>
   Write a Review
  </span>
 </a>
 <div style="width:100%; margin:20px 0px;">
  <b>
   Beer Geek Stats:
  </b>
  <span style="color:#999999;">
   |
   <a class="Tooltip" h

In [16]:
beerstats = soup.find_all(class_='beerstats')

In [17]:
for bs in beerstats:
    print(bs.text)
    print('\n')


Style:
Belgian SaisonRanked #65
ABV:
 5%
Score:
94Ranked #1,194


Style:


Belgian SaisonRanked #65


ABV:


 5%


Score:


94Ranked #1,194


Avg:


4.26 | pDev: 8.22%


Reviews:


93


Ratings:


597


From:


BFM (Brasserie des Franches-Montagnes)


 


Switzerland


Avail:


Rotating


Wants


  32


Gots


  83




In [30]:
list(res.children)[5]

<div style="width:100%; margin:20px 0px;">
<b>Beer Geek Stats:</b>
<span style="color:#999999;"> | <a class="Tooltip" href="/beer/shelf-talker/?b=87869" style="color:#999999;" target="_blank" title="Sell this beer? Increase your sales by using our scores in your marketing and on your shelves.">Print Shelf Talker</a></span>
<dl class="beerstats">
<dt class="beerstats muted"><span class="Tooltip" title="Style category and style ranking (if any)."><b>Style:</b></span></dt>
<dd class="beerstats"><a class="Tooltip" href="/beer/styles/129/" title="Learn more about this style."><b>Belgian Saison</b></a><br/><a class="Tooltip" href="/beer/top-styles/129/" title="Ranking within the style. Click to view the Top Rated Beers for this style.">Ranked #65</a></dd>
<dt class="beerstats muted"><span class="Tooltip" title="Percentage of alcohol by volume."><b>ABV:</b></span></dt>
<dd class="beerstats"> <span class="Tooltip" title="Percentage of alcohol by volume."><b>5%</b></span></dd>
<dt class="beerst

In [32]:
bold = res.find_all('b')

In [33]:
bold

[<b>Beer Geek Stats:</b>,
 <b>Style:</b>,
 <b>Belgian Saison</b>,
 <b>ABV:</b>,
 <b>5%</b>,
 <b>Score:</b>,
 <b>94</b>]

In [57]:
soup.find(title="Learn more about this style.").find('b').text

'Belgian Saison'

In [65]:
soup.find_all(title="Percentage of alcohol by volume.")[-1].find('b').text[:-1]

'5'

In [55]:
soup.find(class_="ba-score Tooltip").text

'94'

In [54]:
soup.find(class_="ba-ravg Tooltip").text

'4.26'

In [60]:
soup.find(title="View more beers and info from this brewery.").text

'BFM (Brasserie des Franches-Montagnes)'

In [64]:
soup.find_all(href=lambda x: x and x.startswith('/place/directory/'))[-1].text

'Switzerland'

In [75]:
soup.find(class_="titleBar")

<div class="titleBar">
<h1>XV (√225 Saison)<br/><span style="color:#999999; font-size:0.75em;">BFM (Brasserie des Franches-Montagnes)</span></h1>
</div>

In [81]:
soup.find('title').text.split('|')[0].strip()

'XV (√225 Saison)'

In [94]:
def parse_beer_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    name = soup.find('title').text.split('|')[0].strip()
    style = soup.find(title="Learn more about this style.").find('b').text
    alc_perc = float(soup.find_all(title="Percentage of alcohol by volume.")[-1].find('b').text[:-1])
    
    score_temp = soup.find(class_="ba-score Tooltip")
    if score_temp is not None:
        score = float(score_temp.text)
    else: score = np.NaN
    rating = float(soup.find(class_="ba-ravg Tooltip").text)
    brewery = soup.find(title="View more beers and info from this brewery.").text
    country = soup.find_all(href=lambda x: x and x.startswith('/place/directory/'))[-1].text
    
    return dict(name=name, style=style, alc_perc=alc_perc, 
                score=score, rating=rating, brewery=brewery, country=country)
    

In [165]:
def get_beer_country(url, session=requests):
    try:
        page = session.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        country = soup.find_all(href=lambda x: x and x.startswith('/place/directory/'))[-1].text
    except:
        country = ''
    return country

In [99]:
parse_beer_page('https://www.beeradvocate.com/beer/profile/2385/87878/')

{'name': 'Dark Ale',
 'style': 'American Brown Ale',
 'alc_perc': 4.0,
 'score': nan,
 'rating': 3.44,
 'brewery': 'Black Duck Brewery',
 'country': 'Australia'}

In [106]:
get_beer_country('https://www.beeradvocate.com/beer/profile/2/3/')

'United States'

In [109]:
countries = []
url = 'https://www.beeradvocate.com/beer/profile/2/{}/'
for i in range(100):
    try:
        countries.append(get_beer_country(url.format(i)))
    except:
        print(i)
    

1
2
22
25
53
57
60
64
70
77
91
98


In [110]:
countries

['United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'Belgium',
 'United States',
 'United States',
 'United States',
 'United States',
 'United Kingdom',
 'United Kingdom',
 'United Kingdom',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'Netherlands',
 'Canada',
 'Canada',
 'Canada',
 'Canada',
 'Canada',
 'United States',
 'United Kingdom',
 'United Kingdom',
 'United States',
 'United States',
 'United States',
 'Canada',
 'Canada',
 'Canada',
 'Canada',
 'Canada',
 'Canada',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United Kingdom',
 'United States',
 'United States',
 'United Kingdom',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States'

In [167]:
s = requests.Session()
timeit.timeit(lambda: get_beer_country(url.format(15002),s), number=5)

3.777793766999821

In [146]:
timeit.timeit(lambda: requests.head(url.format(19)), number=1)

0.8501878259994555

In [147]:
requests.head(url.format(19))

<Response [301]>

In [149]:
1.5* 100000 / 3600

41.666666666666664

## Check parallel requests

In [160]:
import eventlet

urls = [url.format(i) for i in range(50)]
pool = eventlet.GreenPool(50)

countries = []
def pool_work():
    for country in pool.imap(get_beer_country, urls):
        countries.append(country)
        print('one is done')

timeit.timeit(pool_work, number=1)

one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done
one is done


60.20601609700043

In [154]:
urls

['https://www.beeradvocate.com/beer/profile/2/3/',
 'https://www.beeradvocate.com/beer/profile/2/4/',
 'https://www.beeradvocate.com/beer/profile/2/5/',
 'https://www.beeradvocate.com/beer/profile/2/6/',
 'https://www.beeradvocate.com/beer/profile/2/7/',
 'https://www.beeradvocate.com/beer/profile/2/8/',
 'https://www.beeradvocate.com/beer/profile/2/9/',
 'https://www.beeradvocate.com/beer/profile/2/10/',
 'https://www.beeradvocate.com/beer/profile/2/11/',
 'https://www.beeradvocate.com/beer/profile/2/12/',
 'https://www.beeradvocate.com/beer/profile/2/13/',
 'https://www.beeradvocate.com/beer/profile/2/14/',
 'https://www.beeradvocate.com/beer/profile/2/15/',
 'https://www.beeradvocate.com/beer/profile/2/16/',
 'https://www.beeradvocate.com/beer/profile/2/17/',
 'https://www.beeradvocate.com/beer/profile/2/18/',
 'https://www.beeradvocate.com/beer/profile/2/19/',
 'https://www.beeradvocate.com/beer/profile/2/20/',
 'https://www.beeradvocate.com/beer/profile/2/21/',
 'https://www.beera

In [169]:
from concurrent.futures import ThreadPoolExecutor
from requests_futures.sessions import FuturesSession

session = FuturesSession(max_workers=5)
cs = []
timeit.timeit(lambda: cs.append(get_beer_country(url.format(15002),session)), number=5)

0.007156055999985256

In [172]:
cs

['', '', '', '', '']

In [175]:
from concurrent.futures import as_completed
from requests_futures.sessions import FuturesSession

def get_beer_countries_parall(urls):
    countries = []
    session = FuturesSession(max_workers=5)
    
    futures = [session.get(url) for url in urls]
    for future in as_completed(futures):
        try:
            resp = future.result()
            soup = BeautifulSoup(resp.content, 'html.parser')
            country = soup.find_all(href=lambda x: x and x.startswith('/place/directory/'))[-1].text
            countries.append(country)
        except:
            pass
    
    return countries

In [187]:
def get_swiss_beers_parall(urls):
    swissbeers = []
    session = FuturesSession(max_workers=5)
    
    futures = [session.get(url) for url in urls]
    for future in as_completed(futures):
        try:
            resp = future.result()
            soup = BeautifulSoup(resp.content, 'html.parser')
            country = soup.find_all(href=lambda x: x and x.startswith('/place/directory/'))[-1].text
            if country == 'Switzerland':
                swissbeers.append(resp.url)
        except:
            pass
    
    return swissbeers

In [176]:
urls = [url.format(i) for i in range(50)]

timeit.timeit(lambda: get_beer_countries_parall(urls), number=1)

6.967507653000212

In [184]:
session = FuturesSession(max_workers=5)

future = session.get(url.format(19))

In [185]:
resp = future.result()

In [186]:
resp.url

'https://www.beeradvocate.com/beer/profile/9/19/'

In [188]:
swiss_beers_urls = get_swiss_beers_parall([url.format(i) for i in range(1000)])

In [189]:
swiss_beers_urls

[]

In [191]:
460000 * (7/50)

64400.00000000001

In [194]:
30*60/(7/50)

12857.142857142855

In [195]:
swiss_beers_urls_to12k = get_swiss_beers_parall([url.format(i) for i in range(12000)])

KeyboardInterrupt: 

## Login to BeerAdvocate

In [237]:
s = requests.Session()
#url_login = 'https://www.beeradvocate.com/community/login/'
url_login = 'https://www.beeradvocate.com/community/login/login'
with open('ba_credentials','r') as f:
    login, password = f.read().splitlines()

In [238]:
 
payload = {'login': login, 'password': password}
page = s.post(url_login, data=payload)

In [229]:
page.status_code

200

In [219]:
import IPython

IPython.display.HTML(page.content)

TypeError: startswith first arg must be bytes or a tuple of bytes, not str

In [239]:
with open('test.html','bw') as f:
    f.write(page.content)

In [240]:
url_ch = 'https://www.beeradvocate.com/place/list/?c_id=CH&s_id=0&brewery=Y'
ch_page1 = s.get(url_ch)

In [241]:
with open('test2.html','bw') as f:
    f.write(ch_page1.content)

In [242]:
soup_ch1 = BeautifulSoup(ch_page1.content)

In [253]:
list_ch1 = soup_ch1.find_all(href=lambda x: x and x.startswith('/beer/profile/'))
list_ch1[0]['href']

'/beer/profile/41488/'

In [254]:
def brewery_extract(s):
    link = s['href']
    brewery_name = s.text
    return brewery_name, link
    

In [257]:
list_ch1_conv = list(map(brewery_extract, list_ch1))

In [260]:
url_base = 'https://www.beeradvocate.com'
link_brewery = url_base + list_ch1_conv[0][1]

In [263]:
brew_1 = s.get(link_brewery)
brew_1_s = BeautifulSoup(brew_1.content)

In [264]:
list_beers_brew_1 = brew_1_s.find_all(href=lambda x: x and x.startswith('/beer/profile/'))

In [265]:
list_beers_brew_1

[<a href="/beer/profile/41488/?view=beers&amp;show=arc">Archived (5)</a>,
 <a href="/beer/profile/41488/?view=beers&amp;show=all">All</a>,
 <a href="/beer/profile/41488/?view=beers&amp;show=feed">Feed</a>,
 <a href="/beer/profile/41488/219290/"><b>Black Swan Coffee Stout</b></a>,
 <a href="/beer/profile/41488/183565/"><b>Rain Drop Pale Ale</b></a>,
 <a href="/beer/profile/41488/196255/"><b>Retired Bear Amber Ale</b></a>,
 <a href="/beer/profile/41488/187713/"><b>White Swan Amarillo Wheat</b></a>,
 <a href="/beer/profile/41488/#XenForo"><i class="uix_icon uix_icon-jumpToTop"></i> <span class="uix_hide">Top</span></a>]